# Example: Creating a kubernetes namespace

For this demo we are going to be looking at deploying a new namespace to a kubernetes (k8s) cluster. For this we will need to utilize the Terraform [Kubernetes provider](https://registry.terraform.io/providers/hashicorp/kubernetes/latest/docs).

## K8s/namespace_v1/provider.tf
```python
terraform {
  required_providers {
    # Provide the kubernetes provider version to use and where to source it from
    # We will use the provider directly from HashiCorp, but in an air gapped environment you can provide a local source
    kubernetes = {
      source = "hashicorp/kubernetes"
      version = "2.20.0"
    }
  }
}

provider "kubernetes" {
  # Configuration options
  # We only ask/provide a location for the kubeconfig file
  config_path    = var.config_path
}
```

We supplied a variable in `var.config_path` that terraform is going to look for in a variable terraform file.

## K8s/namespace_v1/variables.tf

The variables.tf file is where we define the different variables to use for our deployment. We can define default values to use if none are provided or ommit a default to force the user to provide a value. Enviornmental variables can also be used by appending the variable with `TF_VAR_`

```python
variable "config_path" {
    description = "Path to the kubeconfig file"
    type = string
}

variable "env_label" {
    description = "Pod label to specify development or production resource"
    type = string
    default = "Development"

    validation {
      condition = contains(["Development", "Production"], var.env_label)
      error_message = "Valid values for the environment label are Development or Production"
    }
}

variable "USER" {
    description = "ENVIRONMENTAL VARIABLE: User who requested the resources"
    type = string
}
```

Our top variable is `config_path` and you can see we add a description for what that variable is used for and what type it is. In this case we are looking for a string. There is no default value set as we want the user to provide a location for the correct kubeconfig file to us in deploying a new namespace. This can be provided directly to the command `terraform plan -var config_path=` or if left out terraform will prompt you for input as this variable is not optional. 

The next variable we provide is `env_label` to define whether this a production or a development namespace. The default if no value is provided is `Development` and we then validate that the env_label is set to either `Development` or `Production` and will not allow any other labels for environment. 

The final variable is `USER` and in the description you can see this is called out as an environmental variable. When the variable name is all capital letters in terraform it checks environment variables for a match that starts with `TF_VAR_` in this case we setup 
```bash
export TF_VAR_USER=$USER
```
The username of whoever runs the terraform commands will be supplied in this case. If `TF_VAR_USER` is not defined it will prompt you for a value. You could also provide this variable in the `terraform plan -var USER=` command. We also define the type as a string to match the $USER type

We now have enough to build a namespace resource so let's look at the resource.tf file where the actual resource is defined

## K8s/namespace_v1/resource.tf
```python
resource "kubernetes_namespace_v1" "example" {
  metadata {
    labels = {
      Env = var.env_label
    }

    name = var.USER
  }
}
```

There's not a lot to a namespace resource. We use the environment label defined in the variables.tf file or override it if we want to. The namespace name is also defined and for this use case we are using the local username. This can also be overridden in `terraform plan -var USER=` command

## Create a kubernetes namespace based on the user name ($USER)

If you run the next code block it should create a new namespace for you on the local k3s cluster

In [ ]:
import os
import ipywidgets as widgets
og_dir = os.getcwd()
user_name = os.environ['USER']

def create_k8s_namespace(ev):
    # Check if a user directory already exists
    dir_pres = os.path.exists(og_dir + "/K8s/namespace_v1/" + user_name)
    if dir_pres:
        # We don't need to create it if it does exist
        pass
    else:
        # We create it if it does not exist
        os.mkdir(og_dir + "/K8s/namespace_v1/" + user_name)
    # Copy required terraform files to user directory
    !cp K8s/namespace_v1/*.tf K8s/namespace_v1/{user_name}/
    # Change directory to the namespace_v1 directory
    %cd {og_dir}/K8s/namespace_v1/{user_name}
    with output
        # Initialize terraform and load the required providers
        !terraform init
        # Create the terraform plan and review the changes that are going to be made.
        # Also will prompt you for any undefined but required variables, config_path is provided for a k3s cluster.
        !terraform plan -var config_path="/etc/rancher/k3s/k3s.yaml" -var USER={user_name} -out {user_name}.tfplan
        # Apply the terraform plan and create the resources
        !terraform apply {user_name}.tfplan
        !kubectl get namespace --kubeconfig='/etc/rancher/k3s/k3s.yaml'
    # Change back to the original directory
    %cd {og_dir}
    
button = widgets.Button(description="Create my namespace")
output = widgets.Output()
button.on_click(create_k8s_namespace)
display(button, output)

Our new namespace is now part of the k8s cluster

## Delete the kubernetes namespace tied to the user name ($USER)

If you run the next code block it should delete the namespace tied to the user name who created it. 

In [ ]:
####################################
# THIS WILL DESTROY YOUR NAMESPACE #
####################################
# Change directory to the namespace_v1/$USER directory
%cd K8s
%cd namespace_v1
%cd {os.environ['USER']}
# Destroy the namespace resources on provided kubeconfig cluster. 
!terraform destroy -auto-approve -var config_path="/etc/rancher/k3s/k3s.yaml"
# Change back to the original directory
%cd ..
%cd ..
%cd ..
!kubectl get namespace --kubeconfig='/etc/rancher/k3s/k3s.yaml'